In [1]:
import requests
from dotenv import dotenv_values

# Load the environment variables from the .env file
env_vars = dotenv_values("../local.env")

# Access the environment variables
api_key = env_vars['ALPHA_VANTAGE_API_KEY']

In [35]:
import pandas as pd
import os


def fetchIntradyData(symbol: str, interval: str, month: str, api_key: str) -> pd.DataFrame:
    api_domain = "www.alphavantage.co"
    function = "TIME_SERIES_INTRADAY"
    output_size = "full"

    url = "https://{}/query?function={}&symbol={}&interval={}&month={}&outputsize={}&apikey={}".format(
        api_domain, function, symbol, interval, month, output_size, api_key)

    r = requests.get(url)
    data = r.json()

    price_data = data.get("Time Series (1min)")

    if not price_data:
        print(f"Error: {data.get('Error Message')}")
        return

    # create data frame
    data_frame = pd.DataFrame(
        columns=["Date", "Open", "High", "Low", "Close", "Volume"])

    for date, price in price_data.items():
        data_frame = pd.concat([data_frame, pd.DataFrame([{
            "Date": date,
            "Open": price['1. open'],
            "High": price['2. high'],
            "Low": price['3. low'],
            "Close": price['4. close'],
            "Volume": price['5. volume']
        }])], ignore_index=True)

    data_frame["Date"] = pd.to_datetime(data_frame["Date"])

    return data_frame


def fetchSMAData(symbol: str, interval: str, month: str, time_period: int, api_key: str) -> pd.DataFrame:
    api_domain = "www.alphavantage.co"
    function = "SMA"
    series_type = "close"

    url = "https://{}/query?function={}&month={}&symbol={}&interval={}&time_period={}&series_type={}&apikey={}".format(
        api_domain, function, month, symbol, interval, time_period, series_type, api_key)

    r = requests.get(url)
    data = r.json()

    sma_data = data.get("Technical Analysis: SMA")

    if not sma_data:
        print(f"Error: {data.get('Error Message')}")
        return

    # create data frame
    data_frame = pd.DataFrame(columns=["Date", "SMA_{}".format(time_period)])

    for date, sma in sma_data.items():
        data_frame = pd.concat([data_frame, pd.DataFrame([{
            "Date": date,
            "SMA_{}".format(time_period): sma['SMA']
        }])], ignore_index=True)

    data_frame["Date"] = pd.to_datetime(data_frame["Date"])

    print(data_frame.head())

    return data_frame


def createMergedCSV(symbol: str, interval: str, month: str, api_key: str, result_file: str):
    price_data = fetchIntradyData(symbol, interval, month, api_key)
    sma_10_data = fetchSMAData(symbol, interval, month, 10, api_key)
    sma_50_data = fetchSMAData(symbol, interval, month, 50, api_key)
    sma_100_data = fetchSMAData(symbol, interval, month, 100, api_key)

    # merge
    df = price_data.merge(sma_10_data, how="outer", on="Date")
    df = df.merge(sma_50_data, how="outer", on="Date")
    df = df.merge(sma_100_data, how="outer", on="Date")

    print(df.head())

    if not os.path.exists(result_file):
        df.to_csv(result_file, mode='w', header=True, index=False)
        return

    df.to_csv(result_file, mode='a', header=False, index=False)


def getMonths(start: str, end: str):
    start_year, start_month = map(int, start.split('-'))
    end_year, end_month = map(int, end.split('-'))

    months = []

    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            if year == start_year and month < start_month:
                continue
            if year == end_year and month > end_month:
                break
            months.append(f"{year}-{month:02d}")

    return months

In [36]:
# create csv file
symbol = "AAPL"
interval = "1min"
month = "2021-02"

months = getMonths("2021-01", "2021-02")

result_file = "{}_intraday.csv".format(symbol)
os.remove(result_file) if os.path.exists(result_file) else None

for month in months:
    createMergedCSV(symbol, interval, month, api_key,
                    result_file)

                 Date    SMA_10
0 2021-01-29 19:59:00  129.2480
1 2021-01-29 19:58:00  129.2402
2 2021-01-29 19:57:00  129.2265
3 2021-01-29 19:56:00  129.2148
4 2021-01-29 19:55:00  129.2079
                 Date    SMA_50
0 2021-01-29 19:59:00  129.2594
1 2021-01-29 19:58:00  129.2598
2 2021-01-29 19:57:00  129.2592
3 2021-01-29 19:56:00  129.2604
4 2021-01-29 19:55:00  129.2619
                 Date   SMA_100
0 2021-01-29 19:59:00  129.3029
1 2021-01-29 19:58:00  129.3026
2 2021-01-29 19:57:00  129.3025
3 2021-01-29 19:56:00  129.3032
4 2021-01-29 19:55:00  129.3046
                 Date      Open      High       Low     Close Volume SMA_10  \
0 2021-01-04 04:00:00  130.8340  131.0310  130.5210  131.0110  25026    NaN   
1 2021-01-04 04:01:00  131.0110  131.0310  130.9830  131.0110    798    NaN   
2 2021-01-04 04:02:00  131.1190  131.1390  130.9920  131.0200    580    NaN   
3 2021-01-04 04:03:00  131.0300  131.0510  131.0020  131.0300    218    NaN   
4 2021-01-04 04:04:00  131.28

In [48]:
# get data from csv file
import pandas as pd
data = pd.read_csv(result_file)

# sort data by date and save
data = data.sort_values(by='Date')
data.to_csv(result_file, index=False)
print(data.head())

                      Date     Open     High      Low    Close  Volume
16969  2021-01-04 04:00:00  130.834  131.031  130.521  131.011   25026
16968  2021-01-04 04:01:00  131.011  131.031  130.983  131.011     798
16967  2021-01-04 04:02:00  131.119  131.139  130.992  131.020     580
16966  2021-01-04 04:03:00  131.030  131.051  131.002  131.030     218
16965  2021-01-04 04:04:00  131.286  131.306  131.189  131.236    1492


In [27]:
df = pd.DataFrame(columns=["HELLO", "HI"])
print(df.head())

df_new = pd.DataFrame(columns=["HELLO", "b"])
print(df_new.head())

df.merge(df_new, how="left", on="HELLO")
print(df.head())

Empty DataFrame
Columns: [HELLO, HI]
Index: []
Empty DataFrame
Columns: [HELLO, b]
Index: []
Empty DataFrame
Columns: [HELLO, HI]
Index: []
